In [1]:
%matplotlib
#%matplotlib inline
import os
import csv
import fnmatch
import numpy as np
import datetime
import re 
import pandas as pd
import matplotlib.pyplot as plt
import math
import xlsxwriter
import scipy.stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels
from statsmodels.graphics.regressionplots import abline_plot
from statsmodels.stats.api import anova_lm
import seaborn as sns

pd.options.mode.use_inf_as_na = True

Using matplotlib backend: Qt5Agg


Obtain data in form of pandas dataframe

In [2]:
# csv file with the saved data
fileName = r"C:\DTU\Data\201805_HealthnRehab\data_summary.csv"

df = pd.read_csv(fileName, delimiter=',')

typingMechanismDictKeys = set(df['typing_mechanism'])

# create reference data frame for age bins and gender (0:Less than 30/Male, 1:Greater than 30/Female)
labels = ['age_bins', 'gender', 'typing_mechanism']
dataReference = [['Less than 30', 'Male', 'Dwell-Time'], ['Greater than 30', 'Female', 'Multi-Key Selection']]
df_refAgeGender = pd.DataFrame.from_records(dataReference, columns=labels)

# replace data in df in string or category
df = df.replace({'Less than 30': 0, 'Greater than 30': 1, 'Male': 0, 'Female': 1, 'DT' : 0, 'MS' : 1})

# Create dataframe without Nan values
df_woNaGenderAgeExperience = df.dropna(subset=['gender','age_bins', 'gaze_interaction_experience'], how='any') 

# Create a copy of the dataframe to avoid loc errors.
df_ToAnalyze = df_woNaGenderAgeExperience.copy()

# Convert columns of age_bins, typing_mechanism, gender into categorical columns
df_ToAnalyze['gender'] = pd.Categorical(df_woNaGenderAgeExperience.gender).codes
df_ToAnalyze['typing_mechanism'] = pd.Categorical(df_woNaGenderAgeExperience.typing_mechanism).codes
df_ToAnalyze['age_bins'] = pd.Categorical(df_woNaGenderAgeExperience.age_bins).codes
df_ToAnalyze['gaze_interaction_experience'] = pd.Categorical(df_woNaGenderAgeExperience.gaze_interaction_experience).codes

In [7]:
df_ToAnalyze

,timestamp,subject_name,typing_mechanism,age,age_bins,gender,profession,vision,gaze_interaction_experience,application_of_gaze_interaction_used_before,...,any_suggestions/comments?,dataLog_saved?,comments,typing_speed,ms_per_char,error_rate,read_text_events_frequency_ratio,read_text_events_time_ratio,attended_but_not_selected_rate_time,error_rate_total
1,5/15/2018 14:56,be_DT,0,40,1,1,business,Lenses,1,NaN,...,NaN,Yes,English speaker,1.878433,6.388303,66.275660,0.277778,0.007300,0.042949,68.137830
3,5/15/2018 12:35,KEA_MS,1,20-25,0,0,Student,Normal,1,NaN,...,NaN,Yes,NaN,10.785736,1.112581,13.147724,0.056000,0.003742,0.029752,17.823862
4,5/15/2018 11:11,lone_DT,0,50-55,1,1,nurse,Glasses,1,NaN,...,kom hurtigst mulig igang\n,Yes,NaN,4.103884,2.924059,53.722222,0.205128,0.024690,0.082312,66.146825
5,5/15/2018 12:12,mcc_MS,1,20-25,0,0,student,Normal,1,NaN,...,NaN,Yes,NaN,8.778835,1.366924,40.370370,0.076087,0.005991,0.034549,45.185185
6,5/15/2018 14:44,MK_DT,0,45-50,1,1,Product Developer,Not wearing glasses during exp,1,NaN,...,A brilliant tool for thouse that need it.,Yes,NaN,4.625490,2.594320,18.445341,0.092593,0.005869,0.027000,24.222670
7,5/15/2018 10:58,MT_MS,1,25,0,0,Programmer,Normal,0,"Virtual reality with eye tracking, Eye tracking",...,Backspace,No,NaN,7.144239,1.679675,8.816964,0.185185,0.013547,0.015569,13.634673
8,5/15/2018 12:28,ok_MS,1,20-25,0,0,student,Normal,1,NaN,...,NaN,Yes,NaN,11.135985,1.077588,8.210327,0.045161,0.003269,0.033078,11.105163
9,5/15/2018 12:51,pt_DT,0,27,0,0,Student - Health informatic KU,Lenses,1,NaN,...,NaN,Yes,NaN,6.230274,1.926079,9.676650,0.058442,0.004910,0.013036,22.266896
11,5/15/2018 15:42,sh_MS,1,20-25,0,1,student,Normal,0,"Gaming, hmd typing",...,NaN,Yes,NaN,5.309663,2.260031,25.246753,0.108696,0.001796,0.027771,30.456710
12,5/15/2018 11:48,slh_DT,0,10,0,1,NaN,Normal,1,NaN,...,NaN,No,NaN,5.241627,2.289366,7.530067,0.155556,0.009696,0.020300,22.098367


In [4]:
n_Males = df_ToAnalyze.gender[df_ToAnalyze['gender'] == 0].count()
n_Females = df_ToAnalyze.gender[df_ToAnalyze['gender'] == 1].count()

n_Below30 = df_ToAnalyze.age_bins[df_ToAnalyze['age_bins'] == 0].count()
n_Above30 = df_ToAnalyze.age_bins[df_ToAnalyze['age_bins'] == 1].count()

n_DT = df_ToAnalyze.typing_mechanism[df_ToAnalyze['typing_mechanism'] == 0].count()
n_MS = df_ToAnalyze.typing_mechanism[df_ToAnalyze['typing_mechanism'] == 1].count()

n_ExperienceNever = df_ToAnalyze.gaze_interaction_experience[df_ToAnalyze['gaze_interaction_experience'] == 1].count()
n_ExperienceMultiple = df_ToAnalyze.gaze_interaction_experience[df_ToAnalyze['gaze_interaction_experience'] == 0].count()
n_ExperienceOnce = df_ToAnalyze.gaze_interaction_experience[df_ToAnalyze['gaze_interaction_experience'] == 2].count()


print('Males:', n_Males, 'Females:', n_Females)
print('Below 30:', n_Below30, 'Above 30', n_Above30)
print('Dwell time', n_DT, 'Multi-key selection:', n_MS)
print('No previous experience with gaze:', n_ExperienceNever, 'Tried once before:', n_ExperienceOnce, 'Multiple times', n_ExperienceMultiple)

Males: 16 Females: 13
Below 30: 15 Above 30 14
Dwell time 15 Multi-key selection: 14
No previous experience with gaze: 18 Tried once before: 1 Multiple times 10


In [ ]:
n_DT_Below30 = df_ToAnalyze.typing_mechanism[(df_ToAnalyze['typing_mechanism'] == 0) & (df_ToAnalyze['age_bins'] == 0)].count()
n_DT_Above30 = df_ToAnalyze.typing_mechanism[(df_ToAnalyze['typing_mechanism'] == 0) & (df_ToAnalyze['age_bins'] == 1)].count()

n_MS_Below30 = df_ToAnalyze.typing_mechanism[(df_ToAnalyze['typing_mechanism'] == 1) & (df_ToAnalyze['age_bins'] == 0)].count()
n_MS_Above30 = df_ToAnalyze.typing_mechanism[(df_ToAnalyze['typing_mechanism'] == 1) & (df_ToAnalyze['age_bins'] == 1)].count()


n_DT_Males = df_ToAnalyze.typing_mechanism[(df_ToAnalyze['typing_mechanism'] == 0) & (df_ToAnalyze['gender'] == 0)].count()
n_DT_Females = df_ToAnalyze.typing_mechanism[(df_ToAnalyze['typing_mechanism'] == 0) & (df_ToAnalyze['gender'] == 1)].count()

n_MS_Males = df_ToAnalyze.typing_mechanism[(df_ToAnalyze['typing_mechanism'] == 1) & (df_ToAnalyze['gender'] == 0)].count()
n_MS_Females = df_ToAnalyze.typing_mechanism[(df_ToAnalyze['typing_mechanism'] == 1) & (df_ToAnalyze['gender'] == 1)].count()


print(n_DT_Below30, n_DT_Above30, n_MS_Below30, n_MS_Above30)
print(n_DT_Males, n_DT_Females, n_MS_Males, n_MS_Females)

In [ ]:
# Experience (comfort, challenge level, fun) - 
df_ToAnalyze.how_challenging_was_the_task

# Analysis of typing speed 

### OLS regression

In [ ]:
# OLS regression to check residuals of data for anova testing 
X = df_ToAnalyze[['age_bins', 'typing_mechanism', 'gender']] # independent variable
y_typingSpeed = df_ToAnalyze.typing_speed # dependent variable

model_typingSpeed = sm.OLS(y_typingSpeed, X)
model_fit_typingSpeed = model_typingSpeed.fit()

p_typingSpeed = model_fit_typingSpeed.params

# Plot the residuals of each
residuals_typingSpeed = model_fit_typingSpeed.resid # residuals
fig = sm.qqplot(residuals_typingSpeed)
plt.show()

model_fit_typingSpeed.summary()

Comments:
Only 73.5% of variance is explained -- is high correlation value
Prob(F-statistic) --> Null hypothesis is rejected, that the variability is random and alternate hypothesis that the variability
can be explained is accepted

From the graph, we see that the residuals are normally distributed

### ANOVA 

In [ ]:
formula_typingSpeed = 'df_ToAnalyze.typing_speed ~ df_ToAnalyze.age_bins + df_ToAnalyze.gender + df_ToAnalyze.typing_mechanism + df_ToAnalyze.age_bins:df_ToAnalyze.gender + df_ToAnalyze.age_bins:df_ToAnalyze.typing_mechanism + df_ToAnalyze.gender:df_ToAnalyze.typing_mechanism'
model_typingSpeed = ols(formula_typingSpeed, df_ToAnalyze).fit()
aov_table_typingSpeed = statsmodels.stats.anova.anova_lm(model_typingSpeed, typ=2)
print(aov_table_typingSpeed)

### Plots of typing speed wrt independent variables

In [ ]:
# Box Plots

figBoxPlot = plt.figure()

# Age
ax = figBoxPlot.add_subplot(131)
df_ToAnalyze.boxplot( column = 'typing_speed', by='age_bins', ax = ax, grid = False)
plt.xticks([1, 2], ['Less than 30', 'Greater than 30'])
plt.yticks(np.arange(0, float(df.typing_speed.max())+2, step = 2))
plt.ylabel('Typing speed [in wpm]')

# Gender
ax = figBoxPlot.add_subplot(132)
df_ToAnalyze.boxplot(column = 'typing_speed', by='gender', ax = ax, grid = False)
plt.xticks([1, 2], ['Male', 'Female'])
plt.yticks(np.arange(0, float(df.typing_speed.max())+2, step = 2))
plt.ylabel('Typing speed [in wpm]')

# Typing mechanism
ax = figBoxPlot.add_subplot(133)
df_ToAnalyze.boxplot(column = 'typing_speed', by='typing_mechanism', ax = ax, grid = False)
plt.xticks([1, 2], ['Dwell time', 'MultiKey selection'])
plt.yticks(np.arange(0, float(df.typing_speed.max())+2, step = 2))
plt.ylabel('Typing speed [in wpm]')



In [ ]:
print('Average speed and standard deviation for dwell time selection are:', df_ToAnalyze.typing_speed[df_ToAnalyze['typing_mechanism']==0].mean(), df_ToAnalyze.typing_speed[df_ToAnalyze['typing_mechanism']==0].std())
print('Average speed and standard deviation for multi-key selection are:', df_ToAnalyze.typing_speed[df_ToAnalyze['typing_mechanism']==1].mean(), df_ToAnalyze.typing_speed[df_ToAnalyze['typing_mechanism']==1].std())


In [ ]:
# Histograms
figHist = plt.figure()
bins = np.arange(0, float(df_ToAnalyze.typing_speed.max()+2), step = 2)

# Age
ax = figHist.add_subplot(141)
df_ToAnalyze.hist( column = 'typing_speed', ax = ax, grid = False)
plt.xticks(bins)
plt.xlabel('Typing speed [in wpm]')
plt.yticks(np.arange(0, 11, step = 2))
plt.ylabel('Frequency')

ax = figHist.add_subplot(142)
df_ToAnalyze.typing_speed[df_ToAnalyze['age_bins'] == 0].hist(ax = ax, alpha=0.5, grid = False, label = 'Below 30 years')
#df_ToAnalyze.hist(df_ToAnalyze.typing_speed[df_ToAnalyze['age_bins'] == 0], bins, ax = ax, grid = False)
plt.xticks(bins)
plt.xlabel('Typing speed [in wpm]')
plt.ylabel('Frequency')

ax = figHist.add_subplot(142)
df_ToAnalyze.typing_speed[df_ToAnalyze['age_bins'] == 1].hist(ax = ax, alpha=0.5, grid = False, label = 'Above 30 years')
plt.xticks(bins)
plt.xlabel('Typing speed [in wpm]')
plt.ylabel('Frequency')
plt.title('Typing speed and Age')
plt.legend()

# Gender

ax = figHist.add_subplot(143)
df_ToAnalyze.typing_speed[df_ToAnalyze['gender'] == 0].hist(ax = ax, alpha=0.5, grid = False, label = 'Male')
#df_ToAnalyze.hist(df_ToAnalyze.typing_speed[df_ToAnalyze['age_bins'] == 0], bins, ax = ax, grid = False)
plt.xticks(bins)
plt.xlabel('Typing speed [in wpm]')
plt.ylabel('Frequency')

ax = figHist.add_subplot(143)
df_ToAnalyze.typing_speed[df_ToAnalyze['gender'] == 1].hist(ax = ax, alpha=0.5, grid = False, label = 'Female')
plt.xticks(bins)
plt.xlabel('Typing speed [in wpm]')
plt.ylabel('Frequency')
plt.title('Typing speed and Gender')
plt.legend()

# Typing Mechanism

ax = figHist.add_subplot(144)
df_ToAnalyze.typing_speed[df_ToAnalyze['typing_mechanism'] == 0].hist(ax = ax, alpha=0.5, grid = False, label = 'Dwell time')
#df_ToAnalyze.hist(df_ToAnalyze.typing_speed[df_ToAnalyze['age_bins'] == 0], bins, ax = ax, grid = False)
plt.xticks(bins)
plt.xlabel('Typing speed [in wpm]')
plt.ylabel('Frequency')

ax = figHist.add_subplot(144)
df_ToAnalyze.typing_speed[df_ToAnalyze['typing_mechanism'] == 1].hist(ax = ax, alpha=0.5, grid = False, label = 'Multi-key selection')
plt.xticks(bins)
plt.xlabel('Typing speed [in wpm]')
plt.ylabel('Frequency')
plt.title('Typing speed and Typing Mechanism')
plt.legend()

With the new calculations of typing speed, the histograms for typing speed are normal when distributed as per age and gender, 
but not using typing mechanism. Therefore, mann whitney test will be performed for typing mechanism (their variances are still
more or less equal) and t-test for the others.

### Typing speed statistical analysis

In [ ]:
# Age


# Gender


# Typing mechanism
# the mann whitney test in scipy.stats clearly says that it is reliable only for sample size of 20 in each. Since we do not have
# that, this test will be performed manually, based on - http://psych.unl.edu/psycrs/handcomp/hcmann.PDF
n_DT = df_ToAnalyze.typing_mechanism[df_ToAnalyze['typing_mechanism'] == 0].count()
n_MS = df_ToAnalyze.typing_mechanism[df_ToAnalyze['typing_mechanism'] == 1].count()

# Rank the data and add up
RankedSumDT = sum(df_ToAnalyze.typing_speed[df_ToAnalyze['typing_mechanism'] == 0].rank(axis=0, method='average'))

U_DT = n_DT*n_MS + n_DT*(n_DT + 1)/2 - RankedSumDT
U_MS = n_DT*n_MS - U_DT
U_select = min(U_MS, U_DT)
U_critical = 59
if U_select < U_critical:
    print('reject Ho')

# using the function
U_stat, p_val = scipy.stats.mannwhitneyu(df_ToAnalyze.typing_speed[df_ToAnalyze['typing_mechanism'] == 0], df_ToAnalyze.typing_speed[df_ToAnalyze['typing_mechanism'] == 1])
print(U_stat, p_val)

In [ ]:
print(n_DT, n_MS)

In [ ]:
# Age
a = df_ToAnalyze.typing_speed[df_ToAnalyze['age_bins']==0]
b = df_ToAnalyze.typing_speed[df_ToAnalyze['age_bins']==1]
(h, p) = scipy.stats.kruskal(a.values, b.values)
print('Kruskal result for age: H-statistic=', h, ', pvalue=', p)

# Gender
a = df_ToAnalyze.typing_speed[df_ToAnalyze['gender']==0]
b = df_ToAnalyze.typing_speed[df_ToAnalyze['gender']==1]
(h, p) = scipy.stats.kruskal(a.values, b.values)
print('Kruskal result for gender: H-statistic=', h, ', pvalue=', p)

# Typing mechanism
a = df_ToAnalyze.typing_speed[df_ToAnalyze['typing_mechanism']==0]
b = df_ToAnalyze.typing_speed[df_ToAnalyze['typing_mechanism']==1]
(h, p) = scipy.stats.kruskal(a.values, b.values)
print('Kruskal result for typing mechanism: H-statistic=', h, ', pvalue=', p)

In [ ]:
df_ToAnalyze.error_rate[df_ToAnalyze.typing_mechanism==1].mean()

# Analysis of error rate

### OLS regression

In [3]:
# OLS regression to check residuals of data for anova testing 
X = df_ToAnalyze[['age_bins', 'typing_mechanism', 'gender']] # independent variable
y_errorRate = df_ToAnalyze.error_rate # dependent variable

model_errorRate = sm.OLS(y_errorRate,X)
model_fit_errorRate = model_errorRate.fit()

p_errorRate = model_fit_errorRate.params

# Plot the residuals of each
residuals_errorRate = model_fit_errorRate.resid # residuals
fig = sm.qqplot(residuals_errorRate)
plt.show()

model_fit_errorRate.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             error_rate   R-squared:                       0.557
Model:                            OLS   Adj. R-squared:                  0.506
Method:                 Least Squares   F-statistic:                     10.90
Date:                Sat, 13 Apr 2019   Prob (F-statistic):           8.10e-05
Time:                        17:56:34   Log-Likelihood:                -140.02
No. Observations:                  29   AIC:                             286.0
Df Residuals:                      26   BIC:                             290.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
age_bins            42.6122     11.041      3.859      0.001      19.917      65.307
typing_mechanism    16.8558      9.572      1.761      0.090      -2.820      36.531
gender              -6.0228     11.146     -0.540      0.594     -28.934      16.888
==============================================================================
Omnibus:                       11.372   Durbin-Watson:                   1.955
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               10.231
Skew:                           1.225   Prob(JB):                      0.00600
Kurtosis:                       4.569   Cond. No.                         2.20
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Comments: The residual plot is slightly skewed --> anova cannot be performed 
But Prob(F-statistic) is low enough to reject null hypothesis
Still, anova is not performed due to the skewed residual plot

In [4]:
## TODO: Perform Shierer Ray Hare test 

# For now, it is performed in R

## Equivalence testing of error rate for typing mechansims
Here, the null hypothesis is that error rates for both is not equivalent
For a practical equivalence value of 'del', Ho: |mu1 - mu2| > del and Ha: |mu1 - mu2| < del
So, to reject the null hypothesis, both should be rejected:
mu1 - mu2 > del and mu1 - mu2 < -del
or in terms of two-one sided test :
    ((y1m - y2m) + del)/sigma(1/n1 + 1/n2) > z(1-alpha) and ((y1m - y2m) - del)/sigma(1/n1 + 1/n2) < -z(1-alpha)

In [17]:
sd_errorRate_DT = df_ToAnalyze.error_rate[df_ToAnalyze['typing_mechanism']==0].std()
sd_errorRate_MS = df_ToAnalyze.error_rate[df_ToAnalyze['typing_mechanism']==1].std()

n_DT = df_ToAnalyze.typing_mechanism[df_ToAnalyze['typing_mechanism'] == 0].count()
n_MS = df_ToAnalyze.typing_mechanism[df_ToAnalyze['typing_mechanism'] == 1].count()

mean_errorRate_DT = df_ToAnalyze.error_rate[df_ToAnalyze['typing_mechanism']==0].mean()
mean_errorRate_MS = df_ToAnalyze.error_rate[df_ToAnalyze['typing_mechanism']==1].mean()

sd = math.sqrt(((n_DT - 1)*(sd_errorRate_DT)**2 + (n_MS - 1)*(sd_errorRate_MS)**2)/(n_DT + n_MS - 2))

print(sd*0.3)

9.44440451869594


In [18]:
delError = 10 # As per Lakens 2017 Equivalence testing, del = d*sd, where d is Cohen's d = !!0.3!! - 0.3 is not correct: needs to be the effect size. For the given data, sd = ~36

tL = (mean_errorRate_DT - mean_errorRate_MS - (-delError))/(math.sqrt(((sd_errorRate_DT**2)/n_DT) + (sd_errorRate_MS**2)/n_MS))
tU = (mean_errorRate_DT - mean_errorRate_MS - (delError))/(math.sqrt(((sd_errorRate_DT**2)/n_DT) + (sd_errorRate_MS**2)/n_MS))

#df = ((((sd_errorRate_DT**2)/(n_DT)) + ((sd_errorRate_MS**2)/(n_MS)))**2)/(((sd_errorRate_DT/n_DT)**2)/(n_DT-1) + ((sd_errorRate_MS/n_MS)**2)/(n_MS-1))

df = (n_DT + n_MS - 1)

print('tLower:', tL, ',tUpper:', tU,'with', df, ' degrees of freedom')

tLower: 1.013133563052146 ,tUpper: -0.7341394954984529 with 28  degrees of freedom


#For t(28, 0.05) = 1.701
To reject null hypothesis, both of the following should be followed - 
tU < -t(28, 0.05) AND tL > t(28, 0.05)
As both are not, null hypothesis cannot be rejected 
Therefore, both cannot be equivalent

In [5]:
print('Average error rate and standard deviation for dwell time selection are:', df_ToAnalyze.error_rate[df_ToAnalyze['typing_mechanism']==0].mean(), df_ToAnalyze.error_rate[df_ToAnalyze['typing_mechanism']==0].std())
print('Average error rate and standard deviation for multi-key selection are:', df_ToAnalyze.error_rate[df_ToAnalyze['typing_mechanism']==1].mean(), df_ToAnalyze.error_rate[df_ToAnalyze['typing_mechanism']==1].std())

Average error rate and standard deviation for dwell time selection are: 34.56389169613333 39.505970315291336
Average error rate and standard deviation for multi-key selection are: 32.96715175792857 19.432188494849644


### Plots of error rate wrt independent variables

In [ ]:
fig = plt.figure()

# Age
ax = fig.add_subplot(131)
df_ToAnalyze.boxplot(column = 'error_rate', by='age_bins', ax = ax, grid = False)
plt.xticks([1, 2], ['Less than 30', 'Greater than 30'])
#plt.yticks(np.arange(0, float(df.typing_speed.max())+2, step = 2))
plt.ylabel('Typing speed [in wpm]')

# Gender
ax = fig.add_subplot(132)
df_ToAnalyze.boxplot(column = 'error_rate', by='gender', ax = ax, grid = False)
plt.xticks([1, 2], ['Male', 'Female'])
#plt.yticks(np.arange(0, float(df.typing_speed.max())+2, step = 2))
plt.ylabel('Typing speed [in wpm]')

# Typing mechanism
ax = fig.add_subplot(133)
df_ToAnalyze.boxplot(column = 'error_rate', by='typing_mechanism', ax = ax, grid = False)
plt.xticks([1, 2], ['Dwell time', 'MultiKey selection'])
#plt.yticks(np.arange(0, float(df.typing_speed.max())+2, step = 2))
plt.ylabel('Typing speed [in wpm]')


### Error rate statistical analysis

In [24]:
# Age
a = df_ToAnalyze.error_rate[df_ToAnalyze['age_bins']==0]
b = df_ToAnalyze.error_rate[df_ToAnalyze['age_bins']==1]
(h, p) = scipy.stats.kruskal(a.values, b.values)
print('Kruskal result for age: H-statistic=', h, ', pvalue=', p)

# Typing mechanism
a = df_ToAnalyze.error_rate[df_ToAnalyze['gender']==0]
b = df_ToAnalyze.error_rate[df_ToAnalyze['gender']==1]
(h, p) = scipy.stats.kruskal(a.values, b.values)
print('Kruskal result for gender: H-statistic=', h, ', pvalue=', p)

# Typing mechanism
a = df_ToAnalyze.error_rate[df_ToAnalyze['typing_mechanism']==0]
b = df_ToAnalyze.error_rate[df_ToAnalyze['typing_mechanism']==1]
(h, p) = scipy.stats.kruskal(a.values, b.values)
print('Kruskal result for typing mechanism: H-statistic=', h, ', pvalue=', p)

Kruskal result for age: H-statistic= 5.973333333333329 , pvalue= 0.014523801235818043
Kruskal result for gender: H-statistic= 0.27692307692306883 , pvalue= 0.5987250696528521
Kruskal result for typing mechanism: H-statistic= 0.8399999999999892 , pvalue= 0.35939677082052246


In [ ]:
k = np.hstack(df_ToAnalyze.typing_speed)
plt.hist(k)

In [ ]:
# Plot of error rate and typing speed

In [ ]:
#plt.plot(df_ToAnalyze.typing_speed[df_ToAnalyze.typing_mechanism==0], df_ToAnalyze.error_rate[df_ToAnalyze.typing_mechanism==0], 'bo', label = 'dwell time')
#plt.plot(df_ToAnalyze.typing_speed[df_ToAnalyze.typing_mechanism==1], df_ToAnalyze.error_rate[df_ToAnalyze.typing_mechanism==1], 'ro', label = 'multi-key selection')
n = df_ToAnalyze.subject_name.values 
plt.plot(df_ToAnalyze.typing_speed, df_ToAnalyze.error_rate, 'o')


plt.xlabel('Typing speed [in wpm]')
plt.ylabel('Error rate [in %]')
plt.legend()
for i, txt in enumerate(n):
    plt.annotate(txt, (df_ToAnalyze.typing_speed.values[i], df_ToAnalyze.error_rate.values[i]))
plt.grid('on')

# Total error rate = character error rate + word error rate

## OLS regression

In [9]:
# OLS regression to check residuals of data for anova testing 
X = df_ToAnalyze[['age_bins', 'typing_mechanism', 'gender']] # independent variable
y_errorRate_total = df_ToAnalyze.error_rate_total_normalLevenshtein # dependent variable

model_errorRate_total = sm.OLS(y_errorRate_total,X)
model_fit_errorRate_total = model_errorRate_total.fit()

p_errorRate_total = model_fit_errorRate_total.params

# Plot the residuals of each
residuals_errorRate_total = model_fit_errorRate_total.resid # residuals
fig = sm.qqplot(residuals_errorRate_total)
plt.show()

model_fit_errorRate_total.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                    OLS Regression Results                                    
==============================================================================================
Dep. Variable:     error_rate_total_normalLevenshtein   R-squared:                       0.627
Model:                                            OLS   Adj. R-squared:                  0.584
Method:                                 Least Squares   F-statistic:                     14.57
Date:                                Sat, 13 Apr 2019   Prob (F-statistic):           9.16e-06
Time:                                        17:58:00   Log-Likelihood:                -137.76
No. Observations:                                  29   AIC:                             281.5
Df Residuals:                                      26   BIC:                             285.6
Df Model:                                           3                                         
Covariance Type:                            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
age_bins            42.4376     10.214      4.155      0.000      21.442      63.433
typing_mechanism    15.9217      8.855      1.798      0.084      -2.280      34.124
gender               1.4998     10.311      0.145      0.885     -19.695      22.695
==============================================================================
Omnibus:                        8.240   Durbin-Watson:                   1.881
Prob(Omnibus):                  0.016   Jarque-Bera (JB):                6.522
Skew:                           0.952   Prob(JB):                       0.0384
Kurtosis:                       4.332   Cond. No.                         2.20
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Comments: The residual plot is slightly skewed, as skewness value is not close to 0, and value of kurtosis is high --> anova cannot be performed But Prob(F-statistic) is low enough to reject null hypothesis Still, anova is not performed due to the skewed residual plot

# Equivalence testing of error rate for typing mechansims
Here, the null hypothesis is that error rates for both is not equivalent For a practical equivalence value of 'del', Ho: |mu1 - mu2| > del and Ha: |mu1 - mu2| < del So, to reject the null hypothesis, both should be rejected: mu1 - mu2 > del and mu1 - mu2 < -del or in terms of two-one sided test : ((y1m - y2m) + del)/sigma(1/n1 + 1/n2) > z(1-alpha) and ((y1m - y2m) - del)/sigma(1/n1 + 1/n2) < -z(1-alpha)

In [7]:
sd_errorRate_total_DT = df_ToAnalyze.error_rate_total[df_ToAnalyze['typing_mechanism']==0].std()
sd_errorRate_total_MS = df_ToAnalyze.error_rate_total[df_ToAnalyze['typing_mechanism']==1].std()

n_DT = df_ToAnalyze.typing_mechanism[df_ToAnalyze['typing_mechanism'] == 0].count()
n_MS = df_ToAnalyze.typing_mechanism[df_ToAnalyze['typing_mechanism'] == 1].count()

mean_errorRate_total_DT = df_ToAnalyze.error_rate_total[df_ToAnalyze['typing_mechanism']==0].mean()
mean_errorRate_total_MS = df_ToAnalyze.error_rate_total[df_ToAnalyze['typing_mechanism']==1].mean()

sd = math.sqrt(((n_DT - 1)*(sd_errorRate_total_DT)**2 + (n_MS - 1)*(sd_errorRate_total_MS)**2)/(n_DT + n_MS - 2))

print(sd*0.3)

8.7891756726892


In [8]:
delError = 10 # As per Lakens 2017 Equivalence testing, del = d*sd, where d is Cohen's d = !!0.3!! - 0.3 is not correct: needs to be the effect size. For the given data, sd = ~36

tL = (mean_errorRate_total_DT - mean_errorRate_total_MS - (-delError))/(math.sqrt(((sd_errorRate_total_DT**2)/n_DT) + (sd_errorRate_total_MS**2)/n_MS))
tU = (mean_errorRate_total_DT - mean_errorRate_total_MS - (delError))/(math.sqrt(((sd_errorRate_total_DT**2)/n_DT) + (sd_errorRate_total_MS**2)/n_MS))

#df = ((((sd_errorRate_total_DT**2)/(n_DT)) + ((sd_errorRate_total_MS**2)/(n_MS)))**2)/(((sd_errorRate_total_DT/n_DT)**2)/(n_DT-1) + ((sd_errorRate_total_MS/n_MS)**2)/(n_MS-1))

df = (n_DT + n_MS - 1)

print('tLower:', tL, ',tUpper:', tU,'with', df, ' degrees of freedom')

tLower: 1.3588927052975692 ,tUpper: -0.5096787478436346 with 28  degrees of freedom


#For t(28, 0.05) = 1.701 To reject null hypothesis, both of the following should be followed - tU < -t(28, 0.05) AND tL > t(28, 0.05) As both are not, null hypothesis cannot be rejected Therefore, both cannot be equivalent

In [17]:
print('Average error rate and standard deviation for dwell time selection are:', df_ToAnalyze.error_rate_total[df_ToAnalyze['typing_mechanism']==0].mean(), df_ToAnalyze.error_rate_total[df_ToAnalyze['typing_mechanism']==0].std())
print('Average error rate and standard deviation for multi-key selection are:', df_ToAnalyze.error_rate_total[df_ToAnalyze['typing_mechanism']==1].mean(), df_ToAnalyze.error_rate_total[df_ToAnalyze['typing_mechanism']==1].std())
print('Average error rate and standard deviation are:', df_ToAnalyze.error_rate_total.mean(), df_ToAnalyze.error_rate_total.std())

print('Range of error rate and standard deviation is: ', df_ToAnalyze.error_rate_total.min(), 'to',  df_ToAnalyze.error_rate_total.max())

print('Average error rate and standard deviation (only normal levenshtein distance) are:', df_ToAnalyze.error_rate_total_normalLevenshtein.mean(), df_ToAnalyze.error_rate_total_normalLevenshtein.std())

print('Range of error rate and standard deviation (only normal levenshtein distance) is: ', df_ToAnalyze.error_rate_total_normalLevenshtein.min(), 'to',  df_ToAnalyze.error_rate_total_normalLevenshtein.max())


Average error rate and standard deviation for dwell time selection are: 42.036046376880286 35.28910761283365
Average error rate and standard deviation for multi-key selection are: 37.49132342274147 21.013525087932788
Average error rate and standard deviation are: 39.84204219212362 28.862017687188693
Range of error rate and standard deviation is:  2.535974067046173 to 108.58974358974358
Average error rate and standard deviation (only normal levenshtein distance) are: 37.52299452119481 26.745019745527298
Range of error rate and standard deviation (only normal levenshtein distance) is:  2.0457779886148013 to 95.70512820512822


## Plots of error rate wrt independent variables 

In [22]:
fig = plt.figure()

# Age
ax = fig.add_subplot(131)
df_ToAnalyze.boxplot(column = 'error_rate_total', by='age_bins', ax = ax, grid = False)
plt.xticks([1, 2], ['Less than 30', 'Greater than 30'])
#plt.yticks(np.arange(0, float(df.typing_speed.max())+2, step = 2))
plt.ylabel('Error rate [in %]')

# Gender
ax = fig.add_subplot(132)
df_ToAnalyze.boxplot(column = 'error_rate_total', by='gender', ax = ax, grid = False)
plt.xticks([1, 2], ['Male', 'Female'])
#plt.yticks(np.arange(0, float(df.typing_speed.max())+2, step = 2))
plt.ylabel('Error rate [in %]')

# Typing mechanism
ax = fig.add_subplot(133)
df_ToAnalyze.boxplot(column = 'error_rate_total', by='typing_mechanism', ax = ax, grid = False)
plt.xticks([1, 2], ['Dwell time', 'MultiKey selection'])
#plt.yticks(np.arange(0, float(df.typing_speed.max())+2, step = 2))
plt.ylabel('Error rate [in %]')

Text(0,0.5,'Error rate [in %]')

### Error rate statistical analysis

In [23]:
# Age
a = df_ToAnalyze.error_rate_total[df_ToAnalyze['age_bins']==0]
b = df_ToAnalyze.error_rate_total[df_ToAnalyze['age_bins']==1]
(h, p) = scipy.stats.kruskal(a.values, b.values)
print('Kruskal result for age: H-statistic=', h, ', pvalue=', p)

# Typing mechanism
a = df_ToAnalyze.error_rate_total[df_ToAnalyze['gender']==0]
b = df_ToAnalyze.error_rate_total[df_ToAnalyze['gender']==1]
(h, p) = scipy.stats.kruskal(a.values, b.values)
print('Kruskal result for gender: H-statistic=', h, ', pvalue=', p)

# Typing mechanism
a = df_ToAnalyze.error_rate_total[df_ToAnalyze['typing_mechanism']==0]
b = df_ToAnalyze.error_rate_total[df_ToAnalyze['typing_mechanism']==1]
(h, p) = scipy.stats.kruskal(a.values, b.values)
print('Kruskal result for typing mechanism: H-statistic=', h, ', pvalue=', p)

Kruskal result for age: H-statistic= 7.560000000000002 , pvalue= 0.005967799197956402
Kruskal result for gender: H-statistic= 0.06923076923077076 , pvalue= 0.7924600886080329
Kruskal result for typing mechanism: H-statistic= 0.00761904761904475 , pvalue= 0.9304432630047605


In [26]:
# Plot for typing speed and error rate

#plt.plot(df_ToAnalyze.typing_speed[df_ToAnalyze.typing_mechanism==0], df_ToAnalyze.error_rate[df_ToAnalyze.typing_mechanism==0], 'bo', label = 'dwell time')
#plt.plot(df_ToAnalyze.typing_speed[df_ToAnalyze.typing_mechanism==1], df_ToAnalyze.error_rate[df_ToAnalyze.typing_mechanism==1], 'ro', label = 'multi-key selection')
n = df_ToAnalyze.subject_name.values 
plt.plot(df_ToAnalyze.typing_speed, df_ToAnalyze.error_rate_total, 'o')


plt.xlabel('Typing speed [in wpm]')
plt.ylabel('Error rate [in %]')
plt.legend()
for i, txt in enumerate(n):
    plt.annotate(txt, (df_ToAnalyze.typing_speed.values[i], df_ToAnalyze.error_rate_total.values[i]))
plt.grid('on')

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\matplotlib\cbook\deprecation.py:107: MatplotlibDeprecationWarning: Passing one of 'on', 'true', 'off', 'false' as a boolean is deprecated; use an actual boolean (True/False) instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [13]:
# box chart of typing speed and total error rate
label_fontSize = 60
others_fontSize = 54

fig = plt.figure(figsize=(24,24))
ax1 = fig.add_subplot(1,1,1)
ax2 = ax1.twinx()

#onlyPosErr_t

ax1.bar(1, df_ToAnalyze.typing_speed[df_ToAnalyze.typing_mechanism==0].mean(), yerr = [[0], [df_ToAnalyze.typing_speed[df_ToAnalyze.typing_mechanism==0].std()]], label = 'Dwell time')
ax1.bar(2, df_ToAnalyze.typing_speed[df_ToAnalyze.typing_mechanism==1].mean(), yerr = [[0], [df_ToAnalyze.typing_speed[df_ToAnalyze.typing_mechanism==1].std()]], label = 'Multi-key selection')
ax1.set_ylabel('Typing speed [in wpm]', fontsize = label_fontSize)
ax1.set_yticks(np.arange(0, 11, 2))
ax1.legend(bbox_to_anchor=(1, 0.848), loc = 'lower right', fontsize = label_fontSize)
ax1.tick_params(axis='both', which='major', labelsize= others_fontSize)
#ax1.set_xticks([1.5])
#ax1.set_xticklabels(['Typing speed'])

ax2.bar(5, df_ToAnalyze.error_rate_total_normalLevenshtein[df_ToAnalyze.typing_mechanism==0].mean(), yerr = [[0], [df_ToAnalyze.error_rate_total_normalLevenshtein[df_ToAnalyze.typing_mechanism==0].std()]])
ax2.bar(6, df_ToAnalyze.error_rate_total_normalLevenshtein[df_ToAnalyze.typing_mechanism==1].mean(), yerr = [[0], [df_ToAnalyze.error_rate_total_normalLevenshtein[df_ToAnalyze.typing_mechanism==1].std()]])
ax2.set_ylabel('Error rate [in %]', fontsize = label_fontSize)
plt.xticks([], [])
#ax1.grid(color='b', alpha = 0.1, linestyle='-', linewidth=2)
#ax2.grid(color='b', alpha = 0.1, linestyle='-', linewidth=1.7)
ax2.set_ylim([0, 100])
ax2.tick_params(axis='both', which='major', labelsize=others_fontSize)
plt.tight_layout()
fig.savefig('typingSpeed_errorRateTotalNormalLevenshtein_DTandMS.png', dpi = 300, bbox_to_anchor = (0.95, 0.1))

In [16]:
# box chart of typing speed and error rate
label_fontSize = 98
others_fontSize = 90

fig = plt.figure(figsize=(30,30))
ax1 = fig.add_subplot(1,1,1)
ax2 = ax1.twinx()

#onlyPosErr_t
x1_TypingSpeed = 1
x2_TypingSpeed = 2
ax1.bar(x1_TypingSpeed, df_ToAnalyze.typing_speed[df_ToAnalyze.typing_mechanism==0].mean(), yerr = [[0], [df_ToAnalyze.typing_speed[df_ToAnalyze.typing_mechanism==0].std()]], label = 'Dwell time')
ax1.errorbar(x1_TypingSpeed, df_ToAnalyze.typing_speed[df_ToAnalyze.typing_mechanism==0].mean(), yerr = [[0], [df_ToAnalyze.typing_speed[df_ToAnalyze.typing_mechanism==0].std()]], elinewidth=3, ecolor='k')
ax1.bar(x2_TypingSpeed, df_ToAnalyze.typing_speed[df_ToAnalyze.typing_mechanism==1].mean(), yerr = [[0], [df_ToAnalyze.typing_speed[df_ToAnalyze.typing_mechanism==1].std()]], label = 'Multi-key selection')
ax1.errorbar(x2_TypingSpeed, df_ToAnalyze.typing_speed[df_ToAnalyze.typing_mechanism==1].mean(), yerr = [[0], [df_ToAnalyze.typing_speed[df_ToAnalyze.typing_mechanism==1].std()]], elinewidth=3, ecolor='k')
ax1.set_ylabel('Typing speed [in wpm]', fontsize = label_fontSize)
ax1.set_yticks(np.arange(0, 11, 2))
#ax1.legend(bbox_to_anchor=(1, 0.79), loc = 'lower right', fontsize = others_fontSize, frameon=False)
ax1.legend(bbox_to_anchor=(0.27, 0.802), loc = 'lower left', fontsize = others_fontSize, frameon=False)
ax1.tick_params(axis='both', which='major', labelsize= others_fontSize)
ax1.set_xticks([1.5])
ax1.set_xticklabels(['Typing speed'])
ax1.set_ylim([0, 10.4])

# annotate with significance level
maxTypingSpeed = 9.65
y_TypingSpeed, h, col = maxTypingSpeed, 0.2, 'k'
ax1.plot([x1_TypingSpeed, x1_TypingSpeed, x2_TypingSpeed, x2_TypingSpeed], [y_TypingSpeed, y_TypingSpeed+h, y_TypingSpeed+h, y_TypingSpeed], lw=2, c=col)
#plt.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1.5, c=col)
ax1.text((x1_TypingSpeed+x2_TypingSpeed)*.5, y_TypingSpeed+h+0.02, "p<0.001", ha='center', va='bottom', color=col, fontsize = others_fontSize)


x1_ErrorRate = 5
x2_ErrorRate = 6
ax2.bar(x1_ErrorRate, df_ToAnalyze.error_rate_total_normalLevenshtein[df_ToAnalyze.typing_mechanism==0].mean())
ax2.errorbar(x1_ErrorRate, df_ToAnalyze.error_rate_total_normalLevenshtein[df_ToAnalyze.typing_mechanism==0].mean(), yerr = [[0], [df_ToAnalyze.error_rate_total_normalLevenshtein[df_ToAnalyze.typing_mechanism==0].std()]], elinewidth=3, ecolor='k')
ax2.bar(x2_ErrorRate, df_ToAnalyze.error_rate_total_normalLevenshtein[df_ToAnalyze.typing_mechanism==1].mean())
ax2.errorbar(x2_ErrorRate, df_ToAnalyze.error_rate_total_normalLevenshtein[df_ToAnalyze.typing_mechanism==1].mean(), yerr = [[0], [df_ToAnalyze.error_rate_total_normalLevenshtein[df_ToAnalyze.typing_mechanism==1].std()]], linewidth = 3, ecolor='k')
ax2.set_ylabel('Error rate [in %]', fontsize = label_fontSize)
#plt.xticks([], [])
#ax1.grid(color='b', alpha = 0.1, linestyle='-', linewidth=2)
#ax2.grid(color='b', alpha = 0.1, linestyle='-', linewidth=1.7)
ax2.set_ylim([0, 104])
ax2.tick_params(axis='both', which='major', labelsize=others_fontSize)
ax2.set_xticks([1.5, 5.5])
ax2.set_xticklabels(['Typing speed', 'Error rate'])
ax2.spines['bottom'].set_linewidth(4)
ax2.spines['left'].set_linewidth(4)
ax2.spines['right'].set_linewidth(4)
ax2.spines['top'].set_linewidth(4)

# annotate with significance level
maxErrorRate = 72
y_ErrorRate, h, col = maxErrorRate, 2, 'k'
ax2.plot([x1_ErrorRate, x1_ErrorRate, x2_ErrorRate, x2_ErrorRate], [y_ErrorRate+1.5*h, y_ErrorRate+2.5*h, y_ErrorRate+2.5*h, y_ErrorRate+1.5*h], lw=2, c=col)
#plt.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1.5, c=col)
ax2.text((x1_ErrorRate+x2_ErrorRate)*.5, y_ErrorRate+2.5*h, "n.s.", ha='center', va='bottom', color=col, fontsize = others_fontSize)



plt.tight_layout()
fig.savefig('typingSpeed_errorRateTotalNormalLevenshteinDTandMS.png', dpi = 300, bbox_to_anchor = (0.95, 0.1))


#plt.show()
#plt.ioff()

### Correlation between typing speed and error rate

In [16]:
c = str(df_ToAnalyze['typing_speed'].corr(df_ToAnalyze['error_rate_total']))

fig = plt.figure()
ax1 = fig.add_subplot(1,1,1)
ax2 = ax1.twinx()

a = df_ToAnalyze.typing_speed.sort_values()

i = 0
for ind in a.keys():
    i = i + 1
    ax1.plot(i, df_ToAnalyze['typing_speed'].loc[ind], 'bo')
    ax2.plot(i, df_ToAnalyze['error_rate_total'].loc[ind], 'ro')
    
ax1.set_title('Correlation between typing speed and error rate is: %s' %c)
ax1.set_ylabel('Typing speed [in wpm]', color = 'b')
ax1.set_yticks(np.arange(0, 15, 2))
ax2.set_ylabel('Error rate [in %]', color = 'r')

Text(0,0.5,'Error rate [in %]')

In [ ]:
# box chart of typing speed and error rate
label_fontSize = 60
others_fontSize = 54

fig = plt.figure(figsize=(24,24))
ax1 = fig.add_subplot(1,1,1)
ax2 = ax1.twinx()

#onlyPosErr_t

ax1.bar(1, df_ToAnalyze.typing_speed[df_ToAnalyze.typing_mechanism==0].mean(), yerr = [[0], [df_ToAnalyze.typing_speed[df_ToAnalyze.typing_mechanism==0].std()]], label = 'Dwell time')
ax1.bar(2, df_ToAnalyze.typing_speed[df_ToAnalyze.typing_mechanism==1].mean(), yerr = [[0], [df_ToAnalyze.typing_speed[df_ToAnalyze.typing_mechanism==1].std()]], label = 'Multi-key selection')
ax1.set_ylabel('Typing speed [in wpm]', fontsize = label_fontSize)
ax1.set_yticks(np.arange(0, 11, 2))
ax1.legend(bbox_to_anchor=(1, 0.848), loc = 'lower right', fontsize = label_fontSize)
ax1.tick_params(axis='both', which='major', labelsize= others_fontSize)
#ax1.set_xticks([1.5])
#ax1.set_xticklabels(['Typing speed'])

ax2.bar(5, df_ToAnalyze.error_rate[df_ToAnalyze.typing_mechanism==0].mean(), yerr = [[0], [df_ToAnalyze.error_rate[df_ToAnalyze.typing_mechanism==0].std()]])
ax2.bar(6, df_ToAnalyze.error_rate[df_ToAnalyze.typing_mechanism==1].mean(), yerr = [[0], [df_ToAnalyze.error_rate[df_ToAnalyze.typing_mechanism==1].std()]])
ax2.set_ylabel('Error rate [in %]', fontsize = label_fontSize)
plt.xticks([], [])
#ax1.grid(color='b', alpha = 0.1, linestyle='-', linewidth=2)
#ax2.grid(color='b', alpha = 0.1, linestyle='-', linewidth=1.7)
ax2.set_ylim([0, 100])
ax2.tick_params(axis='both', which='major', labelsize=others_fontSize)
plt.tight_layout()
fig.savefig('typingSpeed_errorRate_DTandMS.png', dpi = 300, bbox_to_anchor = (0.95, 0.1))

In [30]:
# box chart of typing speed and error rate
label_fontSize = 98
others_fontSize = 90

fig = plt.figure(figsize=(30,30))
ax1 = fig.add_subplot(1,1,1)
ax2 = ax1.twinx()

#onlyPosErr_t
x1_TypingSpeed = 1
x2_TypingSpeed = 2
ax1.bar(x1_TypingSpeed, df_ToAnalyze.typing_speed[df_ToAnalyze.typing_mechanism==0].mean(), yerr = [[0], [df_ToAnalyze.typing_speed[df_ToAnalyze.typing_mechanism==0].std()]], label = 'Dwell time')
ax1.errorbar(x1_TypingSpeed, df_ToAnalyze.typing_speed[df_ToAnalyze.typing_mechanism==0].mean(), yerr = [[0], [df_ToAnalyze.typing_speed[df_ToAnalyze.typing_mechanism==0].std()]], elinewidth=3, ecolor='k')
ax1.bar(x2_TypingSpeed, df_ToAnalyze.typing_speed[df_ToAnalyze.typing_mechanism==1].mean(), yerr = [[0], [df_ToAnalyze.typing_speed[df_ToAnalyze.typing_mechanism==1].std()]], label = 'Multi-key selection')
ax1.errorbar(x2_TypingSpeed, df_ToAnalyze.typing_speed[df_ToAnalyze.typing_mechanism==1].mean(), yerr = [[0], [df_ToAnalyze.typing_speed[df_ToAnalyze.typing_mechanism==1].std()]], elinewidth=3, ecolor='k')
ax1.set_ylabel('Typing speed [in wpm]', fontsize = label_fontSize)
ax1.set_yticks(np.arange(0, 11, 2))
#ax1.legend(bbox_to_anchor=(1, 0.79), loc = 'lower right', fontsize = others_fontSize, frameon=False)
ax1.legend(bbox_to_anchor=(0.27, 0.802), loc = 'lower left', fontsize = others_fontSize, frameon=False)
ax1.tick_params(axis='both', which='major', labelsize= others_fontSize)
ax1.set_xticks([1.5])
ax1.set_xticklabels(['Typing speed'])
ax1.set_ylim([0, 10.4])

# annotate with significance level
maxTypingSpeed = 9.65
y_TypingSpeed, h, col = maxTypingSpeed, 0.2, 'k'
ax1.plot([x1_TypingSpeed, x1_TypingSpeed, x2_TypingSpeed, x2_TypingSpeed], [y_TypingSpeed, y_TypingSpeed+h, y_TypingSpeed+h, y_TypingSpeed], lw=2, c=col)
#plt.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1.5, c=col)
ax1.text((x1_TypingSpeed+x2_TypingSpeed)*.5, y_TypingSpeed+h+0.02, "p<0.001", ha='center', va='bottom', color=col, fontsize = others_fontSize)


x1_ErrorRate = 5
x2_ErrorRate = 6
ax2.bar(x1_ErrorRate, df_ToAnalyze.error_rate[df_ToAnalyze.typing_mechanism==0].mean())
ax2.errorbar(x1_ErrorRate, df_ToAnalyze.error_rate[df_ToAnalyze.typing_mechanism==0].mean(), yerr = [[0], [df_ToAnalyze.error_rate[df_ToAnalyze.typing_mechanism==0].std()]], elinewidth=3, ecolor='k')
ax2.bar(x2_ErrorRate, df_ToAnalyze.error_rate[df_ToAnalyze.typing_mechanism==1].mean())
ax2.errorbar(x2_ErrorRate, df_ToAnalyze.error_rate[df_ToAnalyze.typing_mechanism==1].mean(), yerr = [[0], [df_ToAnalyze.error_rate[df_ToAnalyze.typing_mechanism==1].std()]], linewidth = 3, ecolor='k')
ax2.set_ylabel('Error rate [in %]', fontsize = label_fontSize)
#plt.xticks([], [])
#ax1.grid(color='b', alpha = 0.1, linestyle='-', linewidth=2)
#ax2.grid(color='b', alpha = 0.1, linestyle='-', linewidth=1.7)
ax2.set_ylim([0, 104])
ax2.tick_params(axis='both', which='major', labelsize=others_fontSize)
ax2.set_xticks([1.5, 5.5])
ax2.set_xticklabels(['Typing speed', 'Error rate'])
ax2.spines['bottom'].set_linewidth(4)
ax2.spines['left'].set_linewidth(4)
ax2.spines['right'].set_linewidth(4)
ax2.spines['top'].set_linewidth(4)

# annotate with significance level
maxErrorRate = 76
y_ErrorRate, h, col = maxErrorRate, 2, 'k'
ax2.plot([x1_ErrorRate, x1_ErrorRate, x2_ErrorRate, x2_ErrorRate], [y_ErrorRate, y_ErrorRate+h, y_ErrorRate+h, y_ErrorRate], lw=2, c=col)
#plt.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1.5, c=col)
ax2.text((x1_ErrorRate+x2_ErrorRate)*.5, y_ErrorRate+h, "n.s.", ha='center', va='bottom', color=col, fontsize = others_fontSize)



plt.tight_layout()
fig.savefig('typingSpeed_errorRate_DTandMS.png', dpi = 300, bbox_to_anchor = (0.95, 0.1))


#plt.show()
#plt.ioff()

In [ ]:
import pylab
fig = pylab.figure()
#figlegend = pylab.figure(figsize=(3,2))
ax = fig.add_subplot(111)
lines = ax.plot(range(10), pylab.randn(10), range(10), pylab.randn(10))
figlegend.legend(lines, ('Dwell time', 'Multi-key selection'), 'center', prop={'size': 6})
#fig.show()
figlegend.show()
figlegend.savefig('legend.png', dpi = 300)

# Typing Mechanism

## Typing Mechanism and other Independent variables

### Pie charts

In [ ]:
fig = plt.figure()

nSubj = len(df_ToAnalyze)

# Overall
# Gender
ax = fig.add_subplot(131)
df_ToAnalyze.gender.value_counts(sort = False).plot.pie(ax = ax, labels = ['Male', 'Female'], autopct='%1.1f%%')
ax.set_title('Gender distribution \n Total count = ' + str(nSubj))

# Age
ax = fig.add_subplot(132)
df_ToAnalyze.age_bins.value_counts(sort = False).plot.pie(ax = ax, labels = ['Less than 30', 'Greater than 30'], autopct='%1.1f%%')
ax.set_title('Age distribution \n Total count = ' + str(nSubj))

# Typing mechanism
ax = fig.add_subplot(133)
df_ToAnalyze.typing_mechanism.value_counts(sort = False).plot.pie(ax = ax, labels = ['Dwell time', 'MultiKey selection'], autopct='%1.1f%%')
ax.set_title('Typing mechanism distribution \n Total count = ' + str(nSubj))


In [ ]:
fig = plt.figure()

# DWELL TIME
# Gender and Typing mechanism
ax = fig.add_subplot(221)
df_ToAnalyze.gender[df['typing_mechanism']==0].value_counts(sort = False).plot.pie(ax = ax, labels = ['Male', 'Female'], autopct='%1.1f%%')
ax.set_title('Dwell time')

# Age and Typing mechanism
ax = fig.add_subplot(223)
df_ToAnalyze.age_bins[df['typing_mechanism']==0].value_counts(sort = False).plot.pie(ax = ax, labels = ['Less than 30', 'Greater than 30'], autopct='%1.1f%%')

# MULTIKEY SELECTION
# Gender and Typing mechanism
ax = fig.add_subplot(222)
df_ToAnalyze.gender[df['typing_mechanism']==1].value_counts(sort = False).plot.pie(ax = ax, labels = ['Male', 'Female'], autopct='%1.1f%%')
ax.set_title('MultiKey Selection')

# Age and Typing mechanism
ax = fig.add_subplot(224)
df_ToAnalyze.age_bins[df['typing_mechanism']==1].value_counts(sort = False).plot.pie(ax = ax, labels = ['Less than 30', 'Greater than 30'], autopct='%1.1f%%')


The minimum number in the pie charts is 5 (Female, MultiKey Selection). This is the number of subjects that will be choosen 
randomly from every category and the statistical tests will be performed again

# Attended But Not Selected Rate

In [ ]:
# Box Plots

figBoxPlot = plt.figure()

# Typing mechanism
ax = figBoxPlot.add_subplot(111)
df_ToAnalyze.boxplot( column = 'attended_but_not_selected_rate_time', by='typing_mechanism', ax = ax, grid = False)
plt.xticks([1, 2], ['Dwell Time', 'Multi-key Selection'])
plt.ylabel('Attended but not selected ratio of time')

In [ ]:
# Equivalence testing on ansr

sd_ansr_DT = df_ToAnalyze.attended_but_not_selected_rate_time[df_ToAnalyze['typing_mechanism']==0].std()
sd_ansr_MS = df_ToAnalyze.attended_but_not_selected_rate_time[df_ToAnalyze['typing_mechanism']==1].std()

mean_ansr_DT = df_ToAnalyze.attended_but_not_selected_rate_time[df_ToAnalyze['typing_mechanism']==0].mean()
mean_ansr_MS = df_ToAnalyze.attended_but_not_selected_rate_time[df_ToAnalyze['typing_mechanism']==1].mean()

n_DT = df_ToAnalyze.typing_mechanism[df_ToAnalyze['typing_mechanism'] == 0].count()
n_MS = df_ToAnalyze.typing_mechanism[df_ToAnalyze['typing_mechanism'] == 1].count()

sd = math.sqrt(((n_DT - 1)*(sd_ansr_DT)**2 + (n_MS - 1)*(sd_ansr_MS)**2)/(n_DT + n_MS - 2))

print(sd*0.3)

print(mean_ansr_DT, sd_ansr_DT)
print(mean_ansr_MS, sd_ansr_MS)

delError = 0.005 # As per Lakens 2017 Equivalence testing, del = d*sd, where d is Cohen's d = 0.3. For the given data, sd = ~0.006

tL = (mean_ansr_DT - mean_ansr_MS - (-delError))/(math.sqrt(((sd_ansr_DT**2)/n_DT) + (sd_ansr_MS**2)/n_MS))
tU = (mean_ansr_DT - mean_ansr_MS - (delError))/(math.sqrt(((sd_ansr_DT**2)/n_DT) + (sd_ansr_MS**2)/n_MS))

#df = ((((sd_errorRate_DT**2)/(n_DT)) + ((sd_errorRate_MS**2)/(n_MS)))**2)/(((sd_errorRate_DT/n_DT)**2)/(n_DT-1) + ((sd_errorRate_MS/n_MS)**2)/(n_MS-1))

df = (n_DT + n_MS - 1)

print('tLower:', tL, ',tUpper:', tU,'with', df, ' degrees of freedom')

 For t(28, 0.05) = 1.701 To reject null hypothesis, both of the following should be followed : tU < -t(28, 0.05) AND 
tL > t(28, 0.05) , but here, the former is true but the latter isnot. So, the null hypothesis cannot be rejected, and both
cannot be said to be equivalent.


In [ ]:
# Histogram of the two typing mechanisms

# Histograms
figHist = plt.figure()
bins = np.arange(0, float(df_ToAnalyze.attended_but_not_selected_rate_time.max())+0.01, step = 0.01)

# Typing Mechanism

ax = figHist.add_subplot(111)
df_ToAnalyze.attended_but_not_selected_rate_time[df_ToAnalyze['typing_mechanism'] == 0].hist(ax = ax, alpha=0.5, grid = False, label = 'Dwell time')
#df_ToAnalyze.hist(df_ToAnalyze.typing_speed[df_ToAnalyze['age_bins'] == 0], bins, ax = ax, grid = False)
#plt.xticks(bins)
#plt.xlabel('Attended but Not Selected Ratio of Time')
plt.ylabel('Frequency')

ax = figHist.add_subplot(111)
df_ToAnalyze.attended_but_not_selected_rate_time[df_ToAnalyze['typing_mechanism'] == 1].hist(ax = ax, alpha=0.5, grid = False, label = 'Multi-key selection')
plt.xticks(bins)
plt.xlabel('Attended but Not Selected Ratio of Time')
plt.ylabel('Frequency')
plt.title('Typing speed and Attended but Not Selected Ratio')
plt.legend()

 Since the histograms are more or less normal and the means are also more or less the same, t-test can be performed

In [ ]:
scipy.stats.ttest_ind(df_ToAnalyze.attended_but_not_selected_rate_time[df.typing_mechanism == 0], df_ToAnalyze.attended_but_not_selected_rate_time[df.typing_mechanism == 1])

### Correlation between ansr and error rate

In [ ]:

c = str(df_ToAnalyze['attended_but_not_selected_rate_time'].corr(df_ToAnalyze['error_rate']))

fig = plt.figure()
ax1 = fig.add_subplot(1,1,1)
ax2 = ax1.twinx()

a = df_ToAnalyze.attended_but_not_selected_rate_time.sort_values()

i = 0
for ind in a.keys():
    i = i + 1
    ax1.plot(i, df_ToAnalyze['attended_but_not_selected_rate_time'].loc[ind], 'bo')
    ax2.plot(i, df_ToAnalyze['error_rate'].loc[ind], 'ro')
    
ax1.set_title('Correlation between ansr and error rate is: %s' %c)
ax1.set_ylabel('Attended but not selected time ratio', color = 'b')
ax1.set_yticks(np.arange(0,0.13,0.02))
ax2.set_ylabel('Error rate [in %]', color = 'r')